# Generate moment coefficients

The following steps are performed to obtain all the artifacts for force coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data for pressure coefficient

In [1]:
from cfdmod.use_cases.pressure.path_manager import CmPathManager
import pathlib

path_manager = CmPathManager(output_path=pathlib.Path("./output/pressure"))

Read parameters file

In [2]:
from cfdmod.use_cases.pressure.moment.Cm_config import CmCaseConfig

config_path = pathlib.Path("./fixtures/tests/pressure/Cm_params.yaml")
post_proc_cfg = CmCaseConfig.from_file(config_path)

post_proc_cfg

CmCaseConfig(time_scale_conversion=TimeScaleParameters(CST_real=0.56, CST_sim=16.89), bodies={'marquise': BodyDefinition(surfaces=['m1_yp', 'm2_zp', 'm3_zm']), 'lanternim': BodyDefinition(surfaces=['L1_xp', 'L2_yp', 'L3_zp_yp', 'L4_zp_ym', 'L5_ym', 'L6_xm']), 'building': BodyDefinition(surfaces=[])}, moment_coefficient={'measurement_1': CmConfig(statistics=[BasicStatisticModel(stats='mean'), BasicStatisticModel(stats='rms'), BasicStatisticModel(stats='skewness'), BasicStatisticModel(stats='kurtosis'), ParameterizedStatisticModel(stats='min', params=ExtremeAbsoluteParamsModel(method_type='Absolute')), ParameterizedStatisticModel(stats='max', params=ExtremeAbsoluteParamsModel(method_type='Absolute'))], bodies=[MomentBodyConfig(name='marquise', sub_bodies=ZoningModel(x_intervals=[-inf, inf], y_intervals=[-inf, inf], z_intervals=[-inf, inf]), lever_origin=(0.0, 10.0, 10.0))], directions=['x', 'y', 'z'], transformation=TransformationConfig(translation=(0, 0, 0), rotation=(0.0, 0.0, 0.0), fi

Read mesh from file

In [3]:
from lnas import LnasFormat

mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.normalized.lnas")
mesh = LnasFormat.from_file(mesh_path)

mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

((1549, 3), (2915, 3))

Calculate moment coefficient

In [4]:
from cfdmod.logger import logger
from cfdmod.use_cases.pressure.moment.Cm_data import CommonOutput, process_Cm

cp_path = pathlib.Path("./fixtures/tests/pressure/data/cp_t.resampled.h5")

for cfg_label, cfg in post_proc_cfg.moment_coefficient.items():
    logger.info(f"Processing Cm config {cfg_label} ...")

    cm_output_dict: dict[str, CommonOutput] = process_Cm(
        mesh=mesh,
        cfg=cfg,
        cp_path=cp_path,
        bodies_definition=post_proc_cfg.bodies,
        time_scale_factor=post_proc_cfg.time_scale_conversion.time_scale,
    )
    already_saved = False
    for direction_lbl, cm_output in cm_output_dict.items():
        path_manager.direction_label = direction_lbl
        if already_saved:
            cm_output.save_outputs(cfg_label=cfg_label, path_manager=path_manager)
        else:
            cm_output.save_region_info(cfg_label=cfg_label, path_manager=path_manager)
            cm_output.save_outputs(cfg_label=cfg_label, path_manager=path_manager)
            already_saved = True
    logger.info(f"Processed Cm config {cfg_label}!")

[2024-05-21 11:13:27,582] [INFO] - cfdmod - Processing Cm config measurement_1 ... (1346717505.py:7)
[2024-05-21 11:13:29,792] [INFO] - cfdmod - Processed Cm config measurement_1! (1346717505.py:25)
[2024-05-21 11:13:29,793] [INFO] - cfdmod - Processing Cm config measurement_2 ... (1346717505.py:7)
[2024-05-21 11:13:30,362] [INFO] - cfdmod - Processed Cm config measurement_2! (1346717505.py:25)
